In [1]:
import pandas as pd
import vaex as vx
import numpy as np
from datemerge import datemerge
from hyperlink import hyperlink
from to_excel_file import to_excel_file

In [18]:
df = pd.read_excel(r"D:\\Data Kompa\\Hotpot\\Tính Hotpot.xlsx", sheet_name='Data')

In [3]:
sentiment = ['Positive','Neutral','Negative']
main = ['Ashima','KTOP','Mala Gang']
compe = ['Asanoha','Momo Paradise','Dookki','Mala Panda','Yangguofu']
all_brands = main + compe
channel = ['Facebook','Fanpage','Food Review','Forum','News','Tiktok','Youtube','Google Review','Instagram','Threads','Linkedin','E-commerce']

In [16]:
Link = r"D:\\Data Kompa\\Hotpot\\final.xlsx"

In [21]:
#SHEET sentiment_interaction

#sentiment
df[['Sentiment','Labels1','Labels2','Likes','Shares','Comments']] = df[['Sentiment','Labels1','Labels2','Likes','Shares','Comments']].fillna(0)

SOV = df.pivot_table(index='Topic',columns='Sentiment',values='Id',aggfunc='count').fillna(0).reset_index()
SOV.columns = [col[1] if isinstance(col, tuple) else col for col in SOV.columns]
SOV.rename(columns={0: 'Khác'},inplace=True)
SOV['Total'] = SOV[['Positive', 'Neutral', 'Negative', 'Khác']].sum(axis=1)
SOV = SOV[['Topic', 'Positive', 'Neutral', 'Negative', 'Khác', 'Total']]
df['Topic'] = pd.Categorical(df['Topic'], categories=all_brands, ordered=True)
df = df.sort_values('Topic').reset_index(drop=True)
print(SOV)

#Interaction
df['Topic'] = df['Topic'].str.strip()
df_main = df[df['Topic'].isin(main)]
inter = df_main.pivot_table(index='Topic', values=['Comments', 'Shares', 'Likes'], aggfunc='sum').fillna(0).reset_index()
inter = inter[['Topic','Comments','Shares','Likes']]


count_post = df_main.pivot_table(index='Topic',columns='post',values='Id',aggfunc='count').fillna(0).reset_index()
count_post

author = df_main.drop_duplicates(subset=['Topic', 'AuthorId']).pivot_table(index='Topic',values='AuthorId',aggfunc='count').reset_index()
author

interaction = pd.merge(pd.merge(count_post, inter, on='Topic'), author, on='Topic')
print('\n',interaction)

           Topic  Positive  Neutral  Negative     Khác     Total
0        Asanoha     831.0   1013.0       7.0     13.0    1864.0
1         Ashima      90.0   2314.0       8.0    653.0    3065.0
2         Dookki    7974.0  64999.0     262.0  71607.0  144842.0
3           KTOP      14.0    776.0       5.0      0.0     795.0
4      Mala Gang     285.0   1759.0      45.0    303.0    2392.0
5     Mala Panda     435.0    957.0      13.0    239.0    1644.0
6  Momo Paradise       2.0     32.0       0.0      0.0      34.0
7      Yangguofu     195.0    590.0       8.0      0.0     793.0

        Topic  post MXH  post News  Comments  Shares     Likes  AuthorId
0     Ashima    2237.0       91.0    5778.0  1017.0   39207.0       931
1       KTOP     732.0        0.0    2477.0    16.0     899.0        87
2  Mala Gang     485.0        0.0    3625.0  6732.0  213232.0      1141


In [22]:
#SHEET sentiment.channel
sent_chn = df.pivot_table(index=['Topic','Sentiment','POE'],columns=['Channel'],values='Id',aggfunc='count').reset_index().reindex(columns = ['Topic','Sentiment','POE'] + channel).fillna(0)
sent_chn = sent_chn[sent_chn['Sentiment'] != 0]

total_poe = df.pivot_table(index=['Topic','Sentiment','POE'],values='Id',aggfunc='count').reset_index()
total_setiment_topic = df.pivot_table(index=['Topic','Sentiment'],values='Id',aggfunc='count').reset_index()
total_poe = pd.merge(total_poe,total_setiment_topic,on=['Topic','Sentiment'])
total_poe['per_POE_by_sent'] = total_poe['Id_x']/total_poe['Id_y'] 
total_poe = total_poe.drop(columns=['Id_x','Id_y'])

sent_chn = pd.merge(sent_chn,total_poe,on=['Topic','Sentiment','POE'])

sent_chn

,Topic,Sentiment,POE,Facebook,Fanpage,Food Review,Forum,News,Tiktok,Youtube,Google Review,Instagram,Threads,Linkedin,E-commerce,per_POE_by_sent
0,Asanoha,Negative,Earned,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
1,Asanoha,Neutral,Earned,33.0,0.0,0.0,0.0,1.0,729.0,0.0,0.0,4.0,0.0,0.0,0.0,0.757157
2,Asanoha,Neutral,Owned,0.0,234.0,0.0,0.0,0.0,1.0,0.0,0.0,11.0,0.0,0.0,0.0,0.242843
3,Asanoha,Positive,Earned,4.0,0.0,0.0,0.0,0.0,743.0,0.0,0.0,0.0,0.0,0.0,0.0,0.898917
4,Asanoha,Positive,Owned,0.0,84.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.101083
5,Ashima,Negative,Earned,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.500000
6,Ashima,Negative,Owned,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000
7,Ashima,Neutral,Earned,1534.0,0.0,21.0,2.0,85.0,118.0,1.0,0.0,7.0,3.0,0.0,0.0,0.765341
8,Ashima,Neutral,Owned,0.0,497.0,0.0,0.0,0.0,15.0,0.0,0.0,31.0,0.0,0.0,0.0,0.234659
9,Ashima,Positive,Earned,20.0,0.0,0.0,0.0,0.0,35.0,0.0,3.0,0.0,1.0,0.0,0.0,0.655556


In [23]:
#SHEET sentiment_labels
Labels = df.pivot_table(index=['Topic','Labels1','Labels2'],columns='Sentiment',values='Id',aggfunc='count').fillna(0).reset_index()
Labels = Labels[Labels['Labels1'] != 0]
Labels = Labels[['Topic','Labels1','Labels2','Positive','Neutral','Negative']]
Labels

Sentiment,Topic,Labels1,Labels2,Positive,Neutral,Negative
1,Asanoha,Chương trình MKT,Bài đăng quảng bá,280.0,663.0,1.0
2,Asanoha,Chương trình MKT,Hoạt động quảng bá nhà hàng,0.0,6.0,0.0
3,Asanoha,Chương trình MKT,Khai trương chi nhánh mới,15.0,110.0,1.0
4,Asanoha,Chương trình MKT,Món ăn mới / Thức uống mới,3.0,59.0,0.0
5,Asanoha,Chương trình MKT,Trò chơi trên fanpage,62.0,11.0,0.0
...,...,...,...,...,...,...
132,Yangguofu,Vận hành chung,Hệ thống chi nhánh,2.0,0.0,0.0
133,Yangguofu,Vận hành chung,giá,4.0,0.0,0.0
134,Yangguofu,Đồ ăn,Chất lượng,76.0,0.0,0.0
135,Yangguofu,Đồ ăn,Nước sốt,1.0,0.0,0.0


In [24]:
with pd.ExcelWriter(Link, engine='openpyxl') as writer:
    # Write df1 starting at cell A1
    Labels.to_excel(writer, sheet_name='sentiment.interaction', startrow=0, startcol=0, index=False)

REPORT

mainbrand

In [25]:
quar = 'Quý 3 2024'

In [26]:
sent = pd.read_excel(r"D:\\Data Kompa\\Hotpot\\Tính Hotpot.xlsx", sheet_name='Sentiment.Interaction')
chn = pd.read_excel(r"D:\\Data Kompa\\Hotpot\\Tính Hotpot.xlsx", sheet_name='Sentiment.Channel')
lb = pd.read_excel(r"D:\\Data Kompa\\Hotpot\\Tính Hotpot.xlsx", sheet_name='Sentiment.Labels')
#demo = pd.read_excel(r"D:\\Data Kompa\\Hotpot\\HOTPOTquar2.xlsx", sheet_name='Demo')

In [27]:
slide_n_dict = {}

In [43]:
#chn_main = chn[(chn['Quý'] == quar)]
df_list=[]
for i in main:
    df_chn = chn[(chn['Quý'] == quar) & (chn['Brand']==i)]
    overall = df_chn.groupby('Brand')['MXH', 'News', 'Total final'].sum().reset_index()
    #total_overall_df = total_overall.to_frame(name="Total").T
    df_list.append(overall)
    
    
    total_overall = df_chn[['MXH', 'News', 'Total final']].sum()
    sent_chn_o = (
    df_chn.groupby(['Sentiment'])[['MXH', 'News', 'Total final']]
    .sum()
    .div(total_overall)
    .fillna(0)
    .round(3)
    .reindex(sentiment)
    .reset_index())
    df_list.append(sent_chn_o)
    
    
    df_working = df[df['Topic']==i]
    date = datemerge(df_working.groupby(['PublishedDate']).agg({'Id': 'count'}).pivot_table(index='PublishedDate', values='Id', aggfunc='sum'), '2024-07-01', '2024-09-30')
    df_list.append(date)
    
    sent = df_chn.groupby(['Sentiment'])['Total final'].sum().round(0).reindex(sentiment).reset_index()
    df_list.append(sent)
    
    df_lb = lb[(lb['Quý'] == quar) & (lb['Brand']==i)]
    labels = df_lb.groupby(['Labels2'])['Brand','Labels2','Total final'].sum().round(0).sort_values(by='Total final',ascending = False)
    df_list.append(labels)
    

    sent_by_chn = df_chn.pivot_table(index=['Sentiment'], values=channel, aggfunc='sum').round(0)
    sent_by_chn = sent_by_chn.transpose().reindex(channel, columns = sentiment)
    sent_by_chn['Total'] = sent_by_chn.sum(axis=1)
    df_list.append(sent_by_chn)
    
    poe = df_chn.pivot_table(index='Brand',columns='Note',values='Total final',aggfunc=sum).round(0)
    poe['Total'] = poe.sum(axis=1).round(0)
    df_list.append(poe)
    
    for j in ['Paid','Owned','Earned']:
        df_chn_poe = df_chn[df_chn['Note']==j]
        sent_by_chn_poe = df_chn_poe.pivot_table(index=['Sentiment'], values=channel, aggfunc='sum').round(0)
        sent_by_chn_poe['Total_sent'] = sent_by_chn_poe.sum(axis=1)
        sent_by_chn_poe = sent_by_chn_poe.transpose().reindex(channel, columns = sentiment).fillna(0)
        sent_by_chn_poe['Total'] = sent_by_chn_poe.sum(axis=1)
        sent_by_chn_poe.index.name = str(i) + '_' + str(j)
        df_list.append(sent_by_chn_poe)
    
    for k in ['Positive','Negative']:
        labels_sent = df_lb.groupby('Labels2')['Labels2',k].sum().round(0).sort_values(by=k ,ascending = False)
        df_list.append(labels_sent)
    

    lb_chn = df_working.pivot_table(index='Labels2', columns='Channel', values='Id', aggfunc='count').reindex(columns=channel).fillna(0)
    # Calculate row sums for sorting
    row_sums = lb_chn.sum(axis=1)
    lb_chn = lb_chn.loc[row_sums.sort_values(ascending=False).index]
    # Normalize each row by its total sum and round to 3 decimal places
    lb_chn = lb_chn.div(lb_chn.sum(axis=1), axis=0).round(3).reset_index()
    lb_chn = lb_chn.drop(lb_chn[lb_chn["Labels2"] == 0].index).head(10)
    df_list.append(lb_chn)

print(df_list)  

C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_5408\3021311122.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  overall = df_chn.groupby('Brand')['MXH', 'News', 'Total final'].sum().reset_index()
C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_5408\3021311122.py:30: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  labels = df_lb.groupby(['Labels2'])['Brand','Labels2','Total final'].sum().round(0).sort_values(by='Total final',ascending = False)
C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_5408\3021311122.py:53: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  labels_sent = df_lb.groupby('Labels2')['Labels2',k].sum().round(0).sort_values(by=k ,ascending = False)
C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_5408\3021311122.py:

[    Brand     MXH  News  Total final
0  Ashima  2980.0  85.0       3065.0,   Sentiment    MXH  News  Total final
0  Positive  0.038   0.0        0.037
1   Neutral  0.959   1.0        0.960
2  Negative  0.003   0.0        0.003,                Id
PublishedDate    
2024-07-01     29
2024-07-02     35
2024-07-03     31
2024-07-04     26
2024-07-05     51
...            ..
2024-09-26     29
2024-09-27     16
2024-09-28     31
2024-09-29     11
2024-09-30     29

[92 rows x 1 columns],   Sentiment  Total final
0  Positive        114.0
1   Neutral       2943.0
2  Negative          8.0,                              Total final
Labels2                                 
Chính sách / CTKM                 1136.0
Hoạt động quảng bá nhà hàng        780.0
Chương trình tuyển dụng            612.0
Voucher                            351.0
Bài đăng quảng bá                   43.0
Món ăn mới / Thức uống mới          36.0
Chất lượng                          24.0
Độ đa dạng menu                     23.0
Hệ

In [29]:
Ashima = []
KTOP = []
Mala_Gang = []

counts = len(df_list)
df_counts = 0  # Counter for the DataFrames

for df_item in df_list:
    df_counts += 1  # Increment the counter
    
    if df_counts < (counts/3+1):
        Ashima.append(df_item)
        slide_n_dict['Ashima'] = Ashima
    
    elif df_counts < ((counts/3)*2+1):
        KTOP.append(df_item)
        slide_n_dict['KTOP'] = KTOP
        
    else:
        Mala_Gang.append(df_item)
        slide_n_dict['Mala Gang'] = Mala_Gang
        
print(slide_n_dict)

{'Ashima': [    Brand     MXH  News  Total final
0  Ashima  2980.0  85.0       3065.0,   Sentiment    MXH  News  Total final
0  Positive  0.038   0.0        0.037
1   Neutral  0.959   1.0        0.960
2  Negative  0.003   0.0        0.003,                Id
PublishedDate    
2024-07-01     29
2024-07-02     35
2024-07-03     31
2024-07-04     26
2024-07-05     51
...            ..
2024-09-26     29
2024-09-27     16
2024-09-28     31
2024-09-29     11
2024-09-30     29

[92 rows x 1 columns],   Sentiment  Total final
0  Positive        114.0
1   Neutral       2943.0
2  Negative          8.0,                              Total final
Labels2                                 
Chính sách / CTKM                 1136.0
Hoạt động quảng bá nhà hàng        780.0
Chương trình tuyển dụng            612.0
Voucher                            351.0
Bài đăng quảng bá                   43.0
Món ăn mới / Thức uống mới          36.0
Chất lượng                          24.0
Độ đa dạng menu                 

In [30]:
import pandas as pd

def to_excel_file(file_name, dict_sheet_df):

    file_directory = "C" + ":" + "\ ".strip() + "Users" + "\ ".strip() + "Kompa-KTelt056" + "\ ".strip() + "Desktop" \
                     + "\ ".strip() + str(file_name) + ".xlsx"
    with pd.ExcelWriter(file_directory) as writer:
        for slide, dataframe in dict_sheet_df.items():

            if type(dataframe) == list:
                start_column = 0
                for each_df in dataframe:
                    each_df.to_excel(writer, sheet_name=slide, startcol=start_column)
                    start_column += len(each_df.axes[1]) + 3

            if (type(dataframe) == pd.DataFrame) or (type(dataframe) == pd.Series):
                dataframe.to_excel(writer, sheet_name=slide)

In [31]:
to_excel_file('HotpotQ3', slide_n_dict)